In [3]:
import os, json
from pyblock2.driver.core import DMRGDriver, SymmetryTypes
import pandas as pd
import numpy as np
from qb_gsee_benchmark.dmrg_utils import load_mps, max_det_coeff
from pathlib import Path
import urllib.request
from qb_gsee_benchmark.utils import retrieve_fcidump_from_sftp

In [5]:
# General parameters
repository_url = "https://github.com/isi-usc-edu/qb-gsee-benchmark/archive/refs/heads/main.zip"
problem_instance_files_repository_path = (
    "data/problem_instances"
)

In [7]:
# Download problem instance files
repository_filepath = Path("repository.zip")

repository_path = Path("qb-gsee-benchmark-main")
if not repository_path.exists():
    # Download repository
    urllib.request.urlretrieve(repository_url, repository_filepath.name)
    # unzip repository
    os.system(f"unzip {repository_filepath}")
    # remove zip
    os.remove(repository_filepath)

problem_instance_files_path = repository_path / problem_instance_files_repository_path
instance_files = list(problem_instance_files_path.glob('problem_instance.*.json'))

Archive:  repository.zip
fc5abfda95830b9baea63a1f277fad6c4187046b
   creating: qb-gsee-benchmark-main/
   creating: qb-gsee-benchmark-main/.github/
   creating: qb-gsee-benchmark-main/.github/workflows/
  inflating: qb-gsee-benchmark-main/.github/workflows/summary_csv.yml  
  inflating: qb-gsee-benchmark-main/.github/workflows/validate_problem_instance_files.yml  
  inflating: qb-gsee-benchmark-main/.gitignore  
   creating: qb-gsee-benchmark-main/BubbleML/
   creating: qb-gsee-benchmark-main/BubbleML/UI/
  inflating: qb-gsee-benchmark-main/BubbleML/UI/MLFunctionsForUI.py  
  inflating: qb-gsee-benchmark-main/BubbleML/UI/UIMainWindow.py  
  inflating: qb-gsee-benchmark-main/BubbleML/UI/form.ui  
  inflating: qb-gsee-benchmark-main/BubbleML/UI/main.py  
   creating: qb-gsee-benchmark-main/BubbleML/miniML/
   creating: qb-gsee-benchmark-main/BubbleML/miniML/Figures/
  inflating: qb-gsee-benchmark-main/BubbleML/miniML/Figures/nnmf.png  
  inflating: qb-gsee-benchmark-main/BubbleML/miniML/

In [8]:
# Retrieve relevant metadata
metadata = []

for file in instance_files:
    with open(file, 'r') as jf:
        json_data = json.load(jf)
    
    
    for task_data in json_data['tasks']:
        features = dict(task_data['features'])
        for sinfo in task_data['supporting_files']:
            if 'fcidump' in sinfo['instance_data_object_url'] or 'FCIDUMP' in sinfo['instance_data_object_url']:
                features = {**features, **sinfo}
                break

        if not 'instance_data_object_url' in features.keys():
            # Flag instances without FCIDUMP files
            print(f'{task_data["task_uuid"]} does not have an fcidump!')
            print(task_data['supporting_files'])
            print(10*'-')

        features['reference_energy'] = task_data['requirements'].get('reference_energy')
        features['reference_energy_units'] = task_data['requirements'].get('reference_energy_units')
        features['task_uuid'] = task_data['task_uuid']
        metadata.append(features)

metadata = pd.DataFrame(metadata)

In [9]:
# Currently problem_instance json-s use two conventions to specify 
# the number of electrons/orbitals for the problem: (avas_ne, avas_no)
# and (num_electrons, num_oribtals); this will be unified in the future;
# for now one needs to run the following code to do it manually

metadata_0 = pd.DataFrame(metadata.loc[~metadata.avas_ne.isna()])
metadata_1 = pd.DataFrame(metadata.loc[metadata.avas_ne.isna()])
metadata_0.drop(columns=['num_electrons', 'num_orbitals'], inplace=True)
metadata_1.drop(columns=['avas_ne', 'avas_no'], inplace=True)
metadata_0.rename(columns={'avas_ne' : 'num_electrons', 'avas_no' : 'num_orbitals'}, inplace=True)
metadata = pd.concat([metadata_0, metadata_1], ignore_index=True)

In [10]:
# This collects task_uuid-s for which metadata is either incomplete or non-standard
# (i.e., not properly formatted)
task_uuid_incomp = metadata.loc[metadata.num_electrons.isna()]['task_uuid'].to_list()
#print(task_uuid_incomp)

In [11]:
# The following code completes the metadata (see above)

username = 'darpa-qb-zapata' # username to access the sftp server with FCIDUMPs
key_path = '/Users/akunitsa/.ssh/darpa-qb-zapata-key.ppk' # private key 
def process_fcidump(r):
    if r['task_uuid'] in task_uuid_incomp:
        #print(r['task_uuid'])
        #print(r['instance_data_object_url'])
        fci = retrieve_fcidump_from_sftp(r['instance_data_object_url'], username, key_path)
        return (fci['NELEC'], fci['NORB'], fci['MS2'] + 1)
    else:
        return (r['num_electrons'], r['num_orbitals'], r['multiplicity'])
    

metadata[['num_electrons', 'num_orbitals', 'multiplicity']] = metadata.apply(process_fcidump, axis=1, result_type='expand')

Parsing FCIDUMP_d_1.68_b_sto-3g_ne_12.3092dd74-660d-4c7a-9d43-16d1436e084b
Parsing FCIDUMP_d_1.68_b_sto-3g_ne_28.96843098-e69d-4d1f-8a88-5b24826f7390
Parsing FCIDUMP_d_1.68_b_cc-pvdz-dk_ne_12.673dfe91-d90e-4ecd-8560-d6d74de11070
Parsing FCIDUMP_d_1.68_b_cc-pvdz-dk_ne_28.4412b7d6-86db-4616-9dd2-2c32ee02560f
Parsing FCIDUMP_d_2.0_b_sto-3g_ne_12.cdf017c4-7c30-4274-9567-f5e55a6ba857
Parsing FCIDUMP_d_2.0_b_sto-3g_ne_28.d9341cde-90e8-4d1e-beb2-4e40246d31ce
Parsing FCIDUMP_d_2.0_b_cc-pvdz-dk_ne_12.5439a548-153b-4990-be12-f0e33c30fe63
Parsing FCIDUMP_d_2.0_b_cc-pvdz-dk_ne_28.e2995852-9911-4021-a82f-5b14ddb2d5c8
Parsing FCIDUMP_d_3.0_b_sto-3g_ne_12.67acc90d-1a9e-49a7-98a1-8ca643dca272
Parsing FCIDUMP_d_3.0_b_sto-3g_ne_28.01d526fb-276e-4369-85d4-66a553a1d02e
Parsing FCIDUMP_d_3.0_b_cc-pvdz-dk_ne_12.b0eb2de4-1286-4cfb-8724-e5e4bfc7a4f4
Parsing FCIDUMP_d_3.0_b_cc-pvdz-dk_ne_28.2101e6fa-9fb3-4697-98be-25a4e0de2079
Parsing FCIDUMP_g_ring_b_cc-pvdz.b0c72570-cf06-4ecc-ae64-52a65720c849
Parsing FCIDUM

In [20]:
mps_data_dir = Path('../../darpa-qb-dmrg-analysis/data/data_storage/') # Location of MPSs;

# it is assumed that each MPS is assigned a dir postfixed with a corresponding task_uuid, i.e.,
# *_task_uuid; this is used to build a correspondence between solutions and task_uuid-s

list_of_solution_dirs = list(mps_data_dir.glob('*'))
solution_paths = []
for sol_dir in list_of_solution_dirs:
    task_uuid = os.path.basename(sol_dir).split('_')[-1]
    solution_paths.append({'task_uuid' : task_uuid, 'solution_dir' : sol_dir})
sol_paths = pd.DataFrame(solution_paths)
#print(sol_paths.head(1))

In [13]:
# If the list of overlaps exists in the scripts directory - 
#  extract task_uuid-s for the processed MPSs

processed_uuids = []
if os.path.isfile('../scripts/overlaps.csv'):
    data = pd.read_csv('../scripts/overlaps.csv')
    processed_uuids = data['task_uuid'].to_list()

In [22]:
# Compute the overlaps and save occupation lists for the dominant
# CSFs; also, flag the problems for which MPSs are missing

overlap_data = []
missing_solutions = []
for index, row in metadata.iterrows():

    if row['task_uuid'] in processed_uuids:
        continue

    n_electrons = int(row["num_electrons"])
    spin = int(row["multiplicity"]) - 1
    n_cas = int(row["num_orbitals"])
    task_uuid = row["task_uuid"]

    path_to_solution = sol_paths.query(f'task_uuid == "{task_uuid}"') 
    if path_to_solution.empty:
        missing_solutions.append({'task_uuid' : task_uuid, 'molecule' : row['molecule_name']})
        continue

    # Use the following line to adjust the number of threads for the 
    # OpenMP version of the block2 
    driver = DMRGDriver(scratch="./tmp", symm_type=SymmetryTypes.SU2, n_threads=4)
    driver.initialize_system(
        n_sites=int(n_cas),
        n_elec=int(n_electrons),
        spin=int(spin),
    )
    full_dir = path_to_solution['solution_dir'].item()/'mps_storage'
    dmrg_loop_dir_lst = list(full_dir.glob("*"))
    assert len(dmrg_loop_dir_lst) == 1 # we assume there is just one solution per instance
    dmrg_loop_dir = dmrg_loop_dir_lst[0]

    assert os.path.isfile(dmrg_loop_dir/'mps_info.bin')

    ket = load_mps(dmrg_loop_dir)

    coeff, csf = max_det_coeff(driver, ket, cutoff=0.0005)

    overlap_data.append({'task_uuid' : task_uuid,
                         'num_orbitals' : row['num_orbitals'], 
                         'num_electrons' : row['num_electrons'], 
                         'hf_coeff' : np.abs(coeff), 
                         'csf' : csf})
        

In [23]:
missing_solutions_df = pd.DataFrame(missing_solutions)
overlap_data_df = pd.DataFrame(overlap_data)

In [24]:
missing_solutions_df.molecule.unique()

array(['be_cc-pVDZ', 'V1_vtz'], dtype=object)

In [27]:
if not overlap_data_df.empty:
    overlap_data_df.rename(columns={'hf_coeff': 'overlap'}, inplace=True)
    overlap_data_df = overlap_data_df[['overlap', 'task_uuid']]
    # Concatenate with the existing data frame
    overlap_data_df = pd.concat([data, overlap_data_df], axis=0)
    overlap_data_df.to_csv('overlaps.csv', index=False)